In [39]:
import numpy as np

from misc.matrix_geometric_resampling import matrix_geometric_resampling

In [41]:
n = 10

matrix = np.identity(n) / n

def unbiased_estimator(rng: np.random.Generator) -> np.ndarray:
    index = rng.integers(n)

    result = np.zeros((n, n))
    result[index, index] = 1

    return result

rng = np.random.default_rng()

result = matrix_geometric_resampling(rng, 1000000, 0.05, unbiased_estimator)
print(result)
np.tensordot(result, matrix, axes=(([0,1], [1,0])))

[[ 8.7419814   0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          9.7911282   0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         10.31088348  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         12.00446528  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         11.7756641   0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          7.13713434
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   8.76918733  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          7.66449653  0.          0.        ]
 [ 0.          0.          0.          0.          0.   

array(10.07271046)